<a href="https://colab.research.google.com/github/yuu-eguci/flower-stuff-lab/blob/main/yuueguci/mnist_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun 28 23:15:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Google Drive をマウントします。
# NOTE: 左のトコをポチポチやってマウントすることも出来ますが(というかそのほうがラク)
# マウントすることを明示するほうが好みなのでしています。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Keras のバージョンが結構重要な感あります。確認しておきます。
!pip list | grep -e Keras -e tensorflow -e h5py -e Pillow -e opencv-python

h5py                          3.1.0              
Keras                         2.4.3              
Keras-Preprocessing           1.1.2              
opencv-python                 4.1.2.30           
Pillow                        7.1.2              
tensorflow                    2.5.0              
tensorflow-datasets           4.0.1              
tensorflow-estimator          2.5.0              
tensorflow-gcs-config         2.5.0              
tensorflow-hub                0.12.0             
tensorflow-metadata           1.0.0              
tensorflow-probability        0.12.1             


In [ ]:
"""mnist_fitting main script です。
NOTE: 全コードを同じ code block に詰め込んでいます。
      Notebook の標準的な使い方と違うと思いますが、慣れるまでは普段やってる感じで使ってみます。
"""

# NOTE: Colaboratory で module imported but unused とか出す方法あるのかな?
# NOTE: この内容なら、 Colab では pip install 不要です。
import os
import time

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint

# これはサンプルにはない。
from keras.utils.np_utils import to_categorical


class MNISTDataset():
    """MNIST dataset をクラスとして定義しています。
    NOTE: image_shape とか、グローバルスコープで定数で定義するのイヤやん?
    NOTE: インスタンス変数が変わることって無いと思うから、 static class にすべきじゃない?
    """

    def __init__(self):

        # MNIST は 28x28x1(グレースケール)のデータセットです。
        self.image_shape = (28, 28, 1)
        # クラス数は当然 0~9 の10個です。
        self.num_classes = 10

    def get_batch(self):

        # NOTE: このメソッド名はサンプルママです。
        #       強化学習の方式をオンライン学習といい、既存データセットから学習することをバッチ学習という。
        #       「バッチ学習」用のデータだから batch なのかな。

        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        # preprocess により 0~1 の値に変換します。
        # label_data(正解データ)は one-hot ベクトルへ変換します。
        # NOTE: one-hot ベクトルは、ラベルの数(10)と等しい数のベクトルです。
        # NOTE: ここはよくわかっていない。
        x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in
                           [y_train, y_test]]

        return x_train, y_train, x_test, y_test

    def preprocess_for_image(self, data):

        data = data.astype('float32')
        # 画像データを 0~1 の値に変換しています。
        data /= 255
        shape = (data.shape[0],) + self.image_shape  # add dataset length
        data = data.reshape(shape)
        return data

    def preprocess_for_label(self, data):

        # ラベルデータを one-hot ベクトルに変換しています。
        # XXX: one-hot を理解していません。
        return to_categorical(data, self.num_classes)


def lenet(input_shape, num_classes):
    """LeNet を定義します。
    NOTE: LeNet は最初期の CNN です。
    """

    model = Sequential()

    # 畳み込み層を追加します。
    # NOTE: 畳み込む領域5x5、フィルタ20枚、特徴マップのサイズは入力と同じ、
    #       活性化関数 ReLU です。
    # NOTE: input_shape を指定する必要があります。入力を受け取る最初の層なので。
    # NOTE: 入力が28x28の画像なら、出力は28x28x20です。
    model.add(Conv2D(
        20,
        kernel_size=5,
        padding='same',
        input_shape=input_shape,
        activation='relu',
    ))
    # 畳み込み層にはプーリング層を追加。
    # NOTE: 2x2の領域ごとに圧縮するので、上の Conv2D とあわせて
    #       特徴マップは14x14x20になります。
    model.add(MaxPooling2D(
        pool_size=(2, 2)
    ))

    # 畳み込み層追加です。
    # NOTE: 最初の Conv2D よりもフィルタが多いです。
    #       深い層ほどフィルタを増やすことは、 CNN における一般的なテクです。
    # NOTE: 入力は14x14x20の特徴マップです(↑でコメントした条件の場合)。
    #       padding=same なので出力も 14x14x50 です。
    model.add(Conv2D(
        50,
        kernel_size=5,
        padding='same',
        activation='relu')
    )
    # プーリング層追加です。
    # NOTE: 更に小さく、7x7x50になります。
    model.add(MaxPooling2D(
        pool_size=(2, 2)
    ))

    # 特徴マップを全結合層と接続できるようにします。
    # NOTE: 特徴マップ->ベクトルの変換らしい。
    model.add(Flatten())

    # 全結合層です。サイズ500のベクトルを出力します。
    model.add(Dense(
        500,
        activation='relu',
    ))
    # 分類クラスぶんのベクトルを出力します。
    model.add(Dense(
        num_classes,
    ))

    # 活性化関数 softmax により、値を 0~1 の確率値へ変換します。
    # XXX: 全結合層がクラスぶんの値を出力するかと思ったが、そういうわけではなかったか。
    # NOTE: 活性化関数は(多分)Dense の定義に含めることも可能。
    #       (activation='softmax' のように)
    #       でもこうして別途書くこともできる。
    #       おそらく↑の XXX の疑問の答えはこれ。
    model.add(Activation('softmax'))

    return model


class Trainer():
    """Fitting を行うクラスです。
    """

    def __init__(self, model, loss, optimizer):

        self._target = model
        self._target.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=['accuracy'],
        )
        self.verbose = 1
        self.log_dir = '/content/drive/MyDrive/TensorBoardLogs/mnist_fitting'
        self.hdf5_file_name = '/content/drive/MyDrive/hdf5/mnist_fitting.hdf5'

    def train(self, x_train, y_train, batch_size, epochs, validation_split):

        # 前回の実行のログを削除しています。
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir)
        os.mkdir(self.log_dir)

        self._target.fit(
            # Input data, numpy array
            x_train,
            # Target data, numpy array or tensorflow tensor
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=validation_split,
            callbacks=[
                TensorBoard(log_dir=self.log_dir),
                ModelCheckpoint(self.hdf5_file_name, save_best_only=True),
            ],
            verbose=self.verbose,
        )


# MNIST を読み込みます。
mnist_dataset = MNISTDataset()

# LeNet モデルを作成します。
lenet_model = lenet(
    mnist_dataset.image_shape,
    mnist_dataset.num_classes,
)

# MNIST からデータセットを取得します。
x_train, y_train, x_test, y_test = mnist_dataset.get_batch()

# Trainer にデータセットを渡します。
trainer = Trainer(
    lenet_model,
    loss='categorical_crossentropy',
    optimizer=Adam(),
)

# Fitting します。
# NOTE: 毎回学習するのは非常にクソ面倒でいらっしゃいますので保存をしたいですね?
#       Trainer.train でわかるように ModelCheckpoint によって保存しています。
start = time.time()
trainer.train(
    x_train,
    y_train,
    batch_size=128,
    epochs=12,
    validation_split=0.2,
)
print(f'Duration until fitting done: {time.time() - start} sec')

# 結果を閲覧します。
score = lenet_model.evaluate(
    x_test,
    y_test,
    verbose=0,
)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/TensorBoardLogs/mnist_fitting